# SQS Demo
- SQS를 활용한 EC2 기반 이미지 리사이징
- 예제 시나리오
    - 이미지를 업로드하면 썸네일 생성
    - 시연 내용 : SQS 활용, 간단한 CI/CD
- 순서
    - S3에 생성(소스코드 / 이미지 업로드)
    - 시작 템플릿 생성
        - 유저데이터로 S3에서 소스코드 받아오고 빌드 후 어플리케이션 자동 시작
    - SQS 생성 및 S3와 연동
    - EC2가 시작되면서 어플리케이션에서 메세지 폴링 및 이미지 리사이징

## 실습 순서
1. 환경 설정 파일의 Queue URL 업데이트
- SQS 대기열 생성
    - 표준
    - 이름 : demo-image-resize-queue
    - 표시 제한 시간 : 30초
    - 액세스 정책 : policy 스크립트의 SourceArn에 S3 Arn 업데이트
- 대기열에 메세지 입력
    - 버킷 속성 : 이벤트 알림 생성
        - demo-image-resize
        - 접두사 : images/ 
        - 모든 객체 생성 이벤트에 대해
        - 대상 : SQS 대기열 선택
- S3 버킷에 폴더 생성, 이미지 업로드, SQS 콘솔의 메세지 전송 및 수신에서 메세지 폴링
    - ObjectCreated 메세지 확인, 메세지 모두 삭제
- Queue URL 복사해서 환경 설정 파일 업데이트
2. 코드 압축해서 S3에 업로드 (최상단 디렉토리)
    - src, .env, package.json, yarn.lock
3. 유저데이터 편집
    - 버킷명, 리전 업데이트
4. IAM 역할 생성
    - EC2가 사용
    - SQSFULLACCESS, S3FULLACCESS, ec2fullaccess 권한 부여
    - demo-ec2-role-for-sqs-image-resize
5. EC2 시작 템플릿 생성
    - demo-my-sqs-image-resize
    - 이미지 : 아마존 리눅스 2023
    - 타입 : t2.micro
    - 키페어 X
    - 디폴트 보안그룹
    - 고급 세부 정보
        - IAM 프로파일 적용
        - 유저데이터 입력
6. ASG 생성
    - demo-image-resize-asg
    - 시작템플릿 선택
    - 서브넷 모두 선택
    - 원하는 용량 1, 최소 1, 최대2
    - 태그 Name // Demo-SQS-image-resize
7. S3 버킷에 이미지를 여러개 업로드
    - 정상 작동한다면 버킷에 새로운 폴더가 만들어지고 리사이징된 파일들이 저장된다
    - 너무 느리다면 ASG에서 인스턴스를 늘리면 된다
8. 리소스 정리 : ASG에서 용량 0, 버킷 삭제